In [4]:
import numpy as np 
import cv2 as cv
import os

rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')

In [5]:
def escala(imx, escala):
    width = int(imx.shape[1] * escala / 100)
    height = int(imx.shape[0] * escala / 100)
    size = (width, height)
    im = cv.resize(imx, size, interpolation = cv.INTER_AREA)
    return im

In [8]:
cap = cv.VideoCapture(0)
i = 0 
while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro.detectMultiScale(gray, 1.3, 5)
    for(x, y, w, h) in rostros:
        #frame = cv.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
        frame2 = frame[y:y+h, x:x+w]
        #cv.imshow('rostros2', frame2)
        frame2 = cv.resize(frame2, (100,100), interpolation = cv.INTER_AREA)
        cv.imwrite('C:/Proyectos/IA/caras/brenda/brenda'+str(i)+'.png', frame2)
        
    cv.imshow('rostros', frame)
    i=i+1
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()

In [1]:
import cv2 as cv 
import numpy as np 
import os

dataSet = 'C:/Proyectos/IA/caras'
faces = os.listdir(dataSet)
print(faces)

labels = []
facesData = []
label = 0 
for face in faces:
    facePath = os.path.join(dataSet, face)
    for faceName in os.listdir(facePath):
        image_path = os.path.join(facePath, faceName)
        img = cv.imread(image_path, 0)
        if img is not None:  # Verifica si la imagen se ha cargado correctamente
            # Redimensionar la imagen a un tamaño específico
            img = cv.resize(img, (100, 100))  # Ajusta el tamaño según sea necesario
            labels.append(label)
            facesData.append(img)
        else:
            print(f"Error: No se pudo cargar la imagen {image_path}")
    label += 1

print(np.count_nonzero(np.array(labels) == 0))

faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.train(facesData, np.array(labels))
faceRecognizer.write('abimaelEigenface.xml')

['abimael', 'axel', 'brenda', 'dante', 'Kiana', 'leo']
500


In [2]:
import cv2 as cv 
import numpy as np 
import os

faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.read('abimaelEigenface.xml')

dataSet = 'C:/Proyectos/IA/caras'
faces = os.listdir(dataSet)

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False: 
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    for (x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2,  (100,100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)
        if result[1] > 2800:
            cv.putText(frame, '{}'.format(faces[result[0]]), (x, y-25), 2, 1.1, (0,255,0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        else:
            cv.putText(frame, 'Desconocido', (x, y-20), 2, 0.8, (0,0,255), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()